In [1]:
import os, argparse, pickle
import matplotlib.pyplot as plt
import torch
from glob import glob
from utils import *
import models

In [2]:
args = argparse.ArgumentParser()
args.add_argument('--gpus', type=str, default='0')
args.add_argument('--batch', type=int, default=1)
args.add_argument('--len', type=int, default=40)
args.add_argument('--b', type=int, default=40)
args.add_argument('--mode', type=str, default='sj_S')
args.add_argument('--model', type=str, default='CombineAutoencoder')
config = args.parse_known_args()[0]

In [3]:
SR = 8192
WINDOW_SIZE = 500 # us
data_length = config.len
BATCH_SIZE = config.batch
K, m = 8, 8
ls = 128

In [5]:
ABSpath = '/home/skuser/'
path = os.path.join(ABSpath, 'ai_model/pytorch/test_model')
data_path = os.path.join(ABSpath,'data')
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
accel_raw_data = pickle.load(open(os.path.join(data_path,'stationary_accel_data.pickle'),'rb'))
sound_raw_data = pickle.load(open(os.path.join(data_path,'stationary_sound_data.pickle'),'rb'))
transfer_f = np.array(pickle.load(open(os.path.join(data_path,'transfer_f.pickle'),'rb')))
transfer_f = torch.from_numpy(transfer_f).to(device)
transfer_f.requires_grad = False

# accel_data = dataSplit(accel_raw_data, takebeforetime=config.b, data_length=data_length, expand=True)
# sound_data = dataSplit(sound_raw_data, takebeforetime=config.b, data_length=data_length, expand=False)
# name = 'CombineAutoencoder_sj_S_40_40_adam_0.001_decay0.7071/*'
# modelsavepath = sorted(glob(os.path.join(path, 'model_save/'+name)), key=lambda x: float(x.split('/')[-1].split('_')[-1]))[0]
# model = getattr(models, config.model)(accel_data.shape[1] * accel_data.shape[2], sound_data.shape[1] * sound_data.shape[2]).to(device)

In [6]:
accel_raw_data[0].shape

(987136, 12)

In [13]:
def dataSplit(data, takebeforetime, data_length=40, expand=True):
    device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
    # data shape list(25, np(987136, 12)), accel, 주의: 샘플별로 안 섞이게 하기
    # 이걸 자르기, (index, window, channel)
    # data_length = int(hp.audio.sr * hp.audio.win_length / 1000000)
    # import pdb; pdb.set_trace()
    if expand:
        if takebeforetime != 0:
            data = [np.concatenate([np.zeros((takebeforetime, i.shape[1]),dtype='float'), i]) for i in data]
        splited_data = torch.cat([torch.cat([torch.from_numpy(_data[idx-takebeforetime:idx+data_length][np.newaxis, ...]) for idx in range(takebeforetime, len(_data) - data_length)]) for _data in data])
    else:
        # splited_data = torch.cat([torch.cat([torch.from_numpy(_data[idx:idx+data_length][np.newaxis, ...]) for idx in range(len(_data) - data_length]) for _data in data])
        splited_data = torch.cat([torch.cat([torch.from_numpy(_data[idx:idx+data_length][np.newaxis, ...]) for idx in range(len(_data) - data_length)]) for _data in data])
    return splited_data.cpu()

data = [np.reshape(np.arange(100000*12), (100000,12)),np.reshape(np.arange(100000*12), (100000,12))]
dataSplit(data,40,40).shape, dataSplit(data,40,40,False).shape

(torch.Size([199920, 80, 12]), torch.Size([199920, 40, 12]))

In [12]:

1199999 in dataSplit(data,40,40)

False

In [11]:
model.load_state_dict(torch.load(modelsavepath)['model'])

<All keys matched successfully>

In [24]:
len(accel_raw_data)

25

In [21]:
accel_data.shape

torch.Size([525107, 80, 12])

In [22]:
sound_data.shape

torch.Size([526082, 40, 8])

In [12]:
res = []
dataset = makeDataset(accel_data, sound_data, train=False)
dataset_generator = DataLoader(dataset, batch_size=1, shuffle=False)
model.eval()
with torch.no_grad():
    res = torch.cat([conv_with_S(model(data.to(device).transpose(1,2)), transfer_f, device) for data, _ in dataset_generator])

In [13]:
conv_with_S(model(dataset[0][0].unsqueeze(0).to(device).transpose(1,2)), transfer_f, device).shape

torch.Size([1, 40, 8])

In [14]:
torch.cat([i for i in res]).shape

torch.Size([21004280, 8])

In [15]:
torch.cat([i for i in sound_data]).shape

torch.Size([21043280, 8])

In [19]:
res.shape

torch.Size([525107, 40, 8])

In [20]:
sound_data.shape

torch.Size([526082, 40, 8])